IMPORTS

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Activation, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler, LabelEncoder
from jinja2 import Template
import matplotlib.pyplot as plt
import os
!pip install scapy
from scapy.all import rdpcap , IP, TCP


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444326 sha256=56bc00fab32ad5e866f8f027cde1b35eb6b32050a81ec409d6683988a2f74072
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


/usr/local/lib/python3.10/dist-packages/scapy/layers/ipsec.py:512: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,
/usr/local/lib/python3.10/dist-packages/scapy/layers/ipsec.py:516: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,


converting wireshark logs file into csv


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
packets = rdpcap('/content/drive/MyDrive/Study_DFs/network_logs.pcap')

In [5]:
def extract_packet_info(packet):
    packet_info = {
        'time': packet.time,
        'src_ip': packet[IP].src if IP in packet else None,
        'dst_ip': packet[IP].dst if IP in packet else None,
        'src_mac': packet.src ,
        'dst_mac': packet.dst ,
        'src_port': packet[TCP].sport if TCP in packet else None, # Check if TCP layer is present
        'dst_port': packet[TCP].dport if TCP in packet else None, # Check if TCP layer is present
        'protocol': packet[0][1].proto , # Changed 1 to IP
        'length': len(packet),
        'tcp_flags': packet.sprintf('%TCP.flags%') ,
        'ttl': packet[IP].ttl if IP in packet else None,
        'id': packet[IP].id if IP in packet else None,
        'seq': packet[TCP].seq if TCP in packet else None, # Check if TCP layer is present
        'ack': packet[TCP].ack if TCP in packet else None, # Check if TCP layer is present
        'payload': bytes(packet.payload).hex()
    }
    return packet_info

# Извлечение информации из всех пакетов
packet_data = [extract_packet_info(packet) for packet in packets]

# Создание DataFrame из извлеченной информации
df = pd.DataFrame(packet_data)

# Сохранение DataFrame в CSV файл
df.to_csv('output.csv', index=False)

print("Преобразование завершено. Данные сохранены в output.csv")

Преобразование завершено. Данные сохранены в output.csv


In [6]:
df = pd.read_csv('output.csv')
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload
0,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,NaN,NaN,17,110,??,128,9,NaN,NaN,45000060000900008011d0f5ac11086dac1108ff008900...
1,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,NaN,NaN,17,110,??,128,10,NaN,NaN,45000060000a00008011d0f4ac11086dac1108ff008900...
2,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,NaN,NaN,17,97,??,128,12,NaN,NaN,45000053000c00008011d1fcac11086dac110802c3c000...
3,1.548712e+09,172.17.8.2,172.17.8.109,00:21:5e:72:9e:b4,14:fe:b5:d4:15:ca,NaN,NaN,17,164,??,128,14525,NaN,NaN,4500009638bd000080119908ac110802ac11086d0035c3...
4,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,NaN,NaN,17,91,??,128,13,NaN,NaN,4500004d000d00008011d201ac11086dac110802e09400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,A,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...
3686,1.548713e+09,216.239.94.252,172.17.8.109,00:04:9a:58:eb:0d,14:fe:b5:d4:15:ca,443.0,49234.0,6,54,FPA,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...
3687,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,A,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...
3688,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,FA,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...


CLEANING DATA


In [7]:
unique_flags = sorted(set(flag for flags in df['tcp_flags'] for flag in flags if flag != '?'))

print(unique_flags)

['A', 'F', 'P', 'R', 'S']


In [8]:
flag_values = {
    'F': 1,
    'S': 2,
    'R': 4,
    'P': 8,
    'A': 16,
    'Unknown': 32  # Assign a value for Unknown
}

# Function to convert flags to numeric representation
def convert_flags_to_num(flags):
    num_value = 0
    if '??' in flags:
        num_value += flag_values['Unknown']
    for flag in flag_values:
        if flag in flags:
            num_value += flag_values[flag]
    return num_value

# Apply the function to the 'tcp_flags' column
df['tcp_flags'] = df['tcp_flags'].apply(convert_flags_to_num)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload
0,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,NaN,NaN,17,110,32,128,9,NaN,NaN,45000060000900008011d0f5ac11086dac1108ff008900...
1,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,NaN,NaN,17,110,32,128,10,NaN,NaN,45000060000a00008011d0f4ac11086dac1108ff008900...
2,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,NaN,NaN,17,97,32,128,12,NaN,NaN,45000053000c00008011d1fcac11086dac110802c3c000...
3,1.548712e+09,172.17.8.2,172.17.8.109,00:21:5e:72:9e:b4,14:fe:b5:d4:15:ca,NaN,NaN,17,164,32,128,14525,NaN,NaN,4500009638bd000080119908ac110802ac11086d0035c3...
4,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,NaN,NaN,17,91,32,128,13,NaN,NaN,4500004d000d00008011d201ac11086dac110802e09400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...
3686,1.548713e+09,216.239.94.252,172.17.8.109,00:04:9a:58:eb:0d,14:fe:b5:d4:15:ca,443.0,49234.0,6,54,25,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...
3687,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...
3688,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,17,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...


In [9]:

df['src_port_missing'] = df['src_port'].isnull().astype(int)
df['dst_port_missing'] = df['dst_port'].isnull().astype(int)
       # Then fill the original columns with -1
df['src_port'].fillna(-1, inplace=True)
df['dst_port'].fillna(-1, inplace=True)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload,src_port_missing,dst_port_missing
0,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,-1.0,-1.0,17,110,32,128,9,NaN,NaN,45000060000900008011d0f5ac11086dac1108ff008900...,1,1
1,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,-1.0,-1.0,17,110,32,128,10,NaN,NaN,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1
2,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,-1.0,-1.0,17,97,32,128,12,NaN,NaN,45000053000c00008011d1fcac11086dac110802c3c000...,1,1
3,1.548712e+09,172.17.8.2,172.17.8.109,00:21:5e:72:9e:b4,14:fe:b5:d4:15:ca,-1.0,-1.0,17,164,32,128,14525,NaN,NaN,4500009638bd000080119908ac110802ac11086d0035c3...,1,1
4,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,-1.0,-1.0,17,91,32,128,13,NaN,NaN,4500004d000d00008011d201ac11086dac110802e09400...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0
3686,1.548713e+09,216.239.94.252,172.17.8.109,00:04:9a:58:eb:0d,14:fe:b5:d4:15:ca,443.0,49234.0,6,54,25,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0
3687,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0
3688,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,17,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0


In [10]:

df['seq_missing'] = df['seq'].isnull().astype(int)
df['ack_missing'] = df['ack'].isnull().astype(int)
df['seq'].fillna(-1, inplace=True)
df['ack'].fillna(-1, inplace=True)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing
0,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,-1.0,-1.0,17,110,32,128,9,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1
1,1.548712e+09,172.17.8.109,172.17.8.255,14:fe:b5:d4:15:ca,ff:ff:ff:ff:ff:ff,-1.0,-1.0,17,110,32,128,10,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1
2,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,-1.0,-1.0,17,97,32,128,12,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1
3,1.548712e+09,172.17.8.2,172.17.8.109,00:21:5e:72:9e:b4,14:fe:b5:d4:15:ca,-1.0,-1.0,17,164,32,128,14525,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1
4,1.548712e+09,172.17.8.109,172.17.8.2,14:fe:b5:d4:15:ca,00:21:5e:72:9e:b4,-1.0,-1.0,17,91,32,128,13,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0
3686,1.548713e+09,216.239.94.252,172.17.8.109,00:04:9a:58:eb:0d,14:fe:b5:d4:15:ca,443.0,49234.0,6,54,25,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0
3687,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,16,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0
3688,1.548713e+09,172.17.8.109,216.239.94.252,14:fe:b5:d4:15:ca,00:04:9a:58:eb:0d,49234.0,443.0,6,54,17,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0


In [11]:
nan_columns = df.columns[df.isna().any()].tolist()
if len(nan_columns) ==0:
  print("No NaN colums left")
else:
  print("all good")

No NaN colums left


In [12]:
df['src_mac'] = df['src_mac'].apply(lambda x: int(x.replace(':', ''), 16) if x is not None else -1)
df['dst_mac'] = df['dst_mac'].apply(lambda x: int(x.replace(':', ''), 16) if x is not None else -1)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing
0,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,128,9,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1
1,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,128,10,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1
2,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,97,32,128,12,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1
3,1.548712e+09,172.17.8.2,172.17.8.109,143318490804,23084204824010,-1.0,-1.0,17,164,32,128,14525,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1
4,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,91,32,128,13,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0
3686,1.548713e+09,216.239.94.252,172.17.8.109,19769387789,23084204824010,443.0,49234.0,6,54,25,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0
3687,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0
3688,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,17,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0


we add more colums related to ip addr (if the conn is local, etc...)

In [15]:

import ipaddress
company_ips = ['172.17.8.0/24']  # Example company IP ranges

def is_internal_ip(ip):
    if ip is None:
        return False
    ip_obj = ipaddress.ip_address(ip)
    for network in company_ips:
        if ip_obj in ipaddress.ip_network(network):
            return True
    return False

df['internal_conn'] = ((df['src_ip'].apply(is_internal_ip)) & (df['dst_ip'].apply(is_internal_ip))).astype(int)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,ttl,id,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing,internal_conn
0,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,128,9,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1,1
1,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,128,10,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1,1
2,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,97,32,128,12,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1,1
3,1.548712e+09,172.17.8.2,172.17.8.109,143318490804,23084204824010,-1.0,-1.0,17,164,32,128,14525,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1,1
4,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,91,32,128,13,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,128,1644,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0,0
3686,1.548713e+09,216.239.94.252,172.17.8.109,19769387789,23084204824010,443.0,49234.0,6,54,25,128,42450,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0,0
3687,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,128,1645,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0,0
3688,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,17,128,1646,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0,0


add key devices colums (if traffic comes from hubs/gateways etc)

In [16]:
df['dc_conn'] = ((df['src_ip'] == '172.17.8.2') | (df['dst_ip'] == '172.17.8.2')).astype(int)
df['gateway_conn'] = ((df['src_ip'] == '172.17.8.1') | (df['dst_ip'] == '172.17.8.1')).astype(int)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,...,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing,internal_conn,dc_conn,gateway_conn
0,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1,1,0,0
1,1.548712e+09,172.17.8.109,172.17.8.255,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1,1,0,0
2,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,97,32,...,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1,1,1,0
3,1.548712e+09,172.17.8.2,172.17.8.109,143318490804,23084204824010,-1.0,-1.0,17,164,32,...,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1,1,1,0
4,1.548712e+09,172.17.8.109,172.17.8.2,23084204824010,143318490804,-1.0,-1.0,17,91,32,...,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3686,1.548713e+09,216.239.94.252,172.17.8.109,19769387789,23084204824010,443.0,49234.0,6,54,25,...,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0,0,0,0
3687,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3688,1.548713e+09,172.17.8.109,216.239.94.252,23084204824010,19769387789,49234.0,443.0,6,54,17,...,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0


converting ip

In [17]:
df['src_ip'] = df['src_ip'].apply(lambda x: int(ipaddress.IPv4Address(x)) if x is not None else -1)
df['dst_ip'] = df['dst_ip'].apply(lambda x: int(ipaddress.IPv4Address(x)) if x is not None else -1)
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,...,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing,internal_conn,dc_conn,gateway_conn
0,1.548712e+09,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1,1,0,0
1,1.548712e+09,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1,1,0,0
2,1.548712e+09,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,97,32,...,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1,1,1,0
3,1.548712e+09,2886797314,2886797421,143318490804,23084204824010,-1.0,-1.0,17,164,32,...,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1,1,1,0
4,1.548712e+09,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,91,32,...,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,1.548713e+09,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3686,1.548713e+09,3639566076,2886797421,19769387789,23084204824010,443.0,49234.0,6,54,25,...,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0,0,0,0
3687,1.548713e+09,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3688,1.548713e+09,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,17,...,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0


convert time

In [18]:
df['time'] = pd.to_datetime(df['time'], unit='s')
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,...,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing,internal_conn,dc_conn,gateway_conn
0,2019-01-28 21:44:14.050142976,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1,1,0,0
1,2019-01-28 21:44:14.050145024,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,-1.000000e+00,-1.000000e+00,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1,1,0,0
2,2019-01-28 21:44:14.061892096,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,97,32,...,-1.000000e+00,-1.000000e+00,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1,1,1,0
3,2019-01-28 21:44:14.062853120,2886797314,2886797421,143318490804,23084204824010,-1.0,-1.0,17,164,32,...,-1.000000e+00,-1.000000e+00,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1,1,1,0
4,2019-01-28 21:44:14.065444096,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,91,32,...,-1.000000e+00,-1.000000e+00,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,2019-01-28 21:56:41.023358976,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3686,2019-01-28 21:56:41.042967040,3639566076,2886797421,19769387789,23084204824010,443.0,49234.0,6,54,25,...,1.990931e+09,2.878876e+09,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0,0,0,0
3687,2019-01-28 21:56:41.043101184,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,2.878876e+09,1.990931e+09,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3688,2019-01-28 21:56:41.043215872,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,17,...,2.878876e+09,1.990931e+09,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0


convert seq and ack with scaler (to normalize the values)

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['seq', 'ack']] = scaler.fit_transform(df[['seq', 'ack']])
df

,time,src_ip,dst_ip,src_mac,dst_mac,src_port,dst_port,protocol,length,tcp_flags,...,seq,ack,payload,src_port_missing,dst_port_missing,seq_missing,ack_missing,internal_conn,dc_conn,gateway_conn
0,2019-01-28 21:44:14.050142976,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,0.000000,0.000000,45000060000900008011d0f5ac11086dac1108ff008900...,1,1,1,1,1,0,0
1,2019-01-28 21:44:14.050145024,2886797421,2886797567,23084204824010,281474976710655,-1.0,-1.0,17,110,32,...,0.000000,0.000000,45000060000a00008011d0f4ac11086dac1108ff008900...,1,1,1,1,1,0,0
2,2019-01-28 21:44:14.061892096,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,97,32,...,0.000000,0.000000,45000053000c00008011d1fcac11086dac110802c3c000...,1,1,1,1,1,1,0
3,2019-01-28 21:44:14.062853120,2886797314,2886797421,143318490804,23084204824010,-1.0,-1.0,17,164,32,...,0.000000,0.000000,4500009638bd000080119908ac110802ac11086d0035c3...,1,1,1,1,1,1,0
4,2019-01-28 21:44:14.065444096,2886797421,2886797314,23084204824010,143318490804,-1.0,-1.0,17,91,32,...,0.000000,0.000000,4500004d000d00008011d201ac11086dac110802e09400...,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,2019-01-28 21:56:41.023358976,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,0.673848,0.466010,45000028066c4000800607faac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3686,2019-01-28 21:56:41.042967040,3639566076,2886797421,19769387789,23084204824010,443.0,49234.0,6,54,25,...,0.466010,0.673848,45000028a5d200008006a893d8ef5efcac11086d01bbc0...,0,0,0,0,0,0,0
3687,2019-01-28 21:56:41.043101184,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,16,...,0.673848,0.466010,45000028066d4000800607f9ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0
3688,2019-01-28 21:56:41.043215872,2886797421,3639566076,23084204824010,19769387789,49234.0,443.0,6,54,17,...,0.673848,0.466010,45000028066e4000800607f8ac11086dd8ef5efcc05201...,0,0,0,0,0,0,0


now proceed with model definition

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
tfd = tfp.distributions
class MDNRNNModel:
  def __init__(self, input_shape, output_shape, num_mixtures, lstm_units, learning_rate=0.001,
                 optimizer='adam', loss_function=None, num_lstm_layers=1, dropout=0.2):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.num_mixtures = num_mixtures
        self.lstm_units = lstm_units
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.num_lstm_layers = num_lstm_layers
        self.dropout = dropout
        self.model = self.build_model()
        self.model.compile(optimizer=self.get_optimizer(), loss=self.loss_function)

  def get_optimizer(self):
        if self.optimizer.lower() == 'adam':
            return Adam(learning_rate=self.learning_rate)
        # Add more optimizers if needed
        else:
            return Adam(learning_rate=self.learning_rate)  # Default to Adam

  def build_model(self):
        model = Sequential()
        for _ in range(self.num_lstm_layers - 1):
            model.add(LSTM(self.lstm_units, return_sequences=True, input_shape=self.input_shape))
            model.add(Dropout(self.dropout))
        model.add(LSTM(self.lstm_units, input_shape=self.input_shape))
        model.add(Dropout(self.dropout))
        model.add(Dense(self.num_mixtures * (2 * self.output_shape + 1))) # Calculate output layer size

        return model
